In [72]:
from traffic_data import METRLADatasetLoader # descarga de datos
import numpy as np # manipulacion de datos
import matplotlib.pyplot as plt # graficar
import torch # crear modelos
from torch.utils.data import TensorDataset, DataLoader # manipular dataset
import torch.nn as nn # para usar las capas predefinidas en torch
import torch.optim as optim # para poder utilizar adam como el optimizador
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.data import Data, Batch

In [73]:
# cargamos los datos
loader = METRLADatasetLoader() 
adj, weig, x, y = loader.get_dataset(num_timesteps_in=12, 
                                     num_timesteps_out=1)

In [74]:
# omitiremos la variable del tiempo
x = [i[:, 0, :] for i in x]

In [75]:
adj = torch.tensor(adj)

In [76]:
N = len(x)
print(len(x), len(y)) # instancias para entrenamiento
print(x[0].shape) # cada instancia tiene 207 nodos en 12 momentos
print(y[0].shape) # el ground truth es el grafo en el siguiente momento

34260 34260
(207, 12)
(207, 1)


In [77]:
# convertimos la lista a un tensor aumentando una dimension mas
# primero convertimos a array porque es mas eficiente 
X = torch.tensor(np.array(x)).permute(0, 2, 1)
Y = torch.tensor(np.array(y)).permute(0, 2, 1).squeeze()
print(X.shape)
print(Y.shape)

torch.Size([34260, 12, 207])
torch.Size([34260, 207])


In [78]:
# particionamos en train y test y cargamos los datos en objetos
# DataLoader para mejorar la eficiencia

train_p = 0.8 # porcentaje de training
batch_size = 128

train_size = int(train_p * len(X))
test_size = len(X) - train_size

X_train, X_test = torch.split(X, [train_size, test_size])
Y_train, Y_test = torch.split(Y, [train_size, test_size])

train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_test, Y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                          shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, 
                         shuffle=False, drop_last=True)

In [79]:
class LSTMGATmodel(nn.Module):
    def __init__(self, n_nodes, rnn_hidden_size, 
                 fc_hidden_size, feat_per_node, gat_hidden_size, 
                 att_heads, 
                 batch_size, seq_len, num_lstm_layers=1):
        super().__init__()
        # parametros
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.n_nodes = n_nodes
        
        # capa espacial
        self.gat1 = GATConv(feat_per_node, gat_hidden_size, 
                            heads=att_heads)
        self.gat2 = GATConv(gat_hidden_size * att_heads, 1, 
                            heads=1, concat=False)

        # capa temporal
        self.rnn = nn.LSTM(n_nodes, rnn_hidden_size, 
                           batch_first=True, num_layers=num_lstm_layers)
        self.fc1 = nn.Linear(rnn_hidden_size, fc_hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden_size, n_nodes)

    def forward(self, batch):
        # capa espacial
        out = self.gat1(batch.x, batch.edge_index)
        out = F.elu(out)
        out = self.gat2(out, batch.edge_index)
        out = F.elu(out)
        
        # reestructuracion de datos
        out = out.view(self.batch_size, self.seq_len, self.n_nodes)
        
        # capa temporal
        out, (hidden, cell) = self.rnn(out)
        out = hidden[-1, :, :] 
        out = self.fc1(out) 
        out = self.relu(out) 
        out = self.fc2(out) 
        return out

In [80]:
# inicializamos los parametros del modelo
n_nodes = 207
rnn_hidden_size = n_nodes * 2
fc_hidden_size = rnn_hidden_size * 2
feat_per_node = 1
gat_hidden_size = 2
att_heads = 1
batch_size = 128
seq_len = 12
num_lstm_layers = 1

In [81]:
model = LSTMGATmodel(n_nodes, rnn_hidden_size, 
                 fc_hidden_size, feat_per_node, gat_hidden_size, 
                 att_heads,
                 batch_size, seq_len, num_lstm_layers=1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(device)

cuda


In [82]:
# Definición de la función de pérdida y el optimizador
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
lossi = []

# Entrenamiento del modelo
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    for X_batch, Y_batch in train_loader:
        Xcompact = X_batch.view(seq_len * batch_size, n_nodes)
        grafos = [Data(i.view(-1, 1), adj) for i in Xcompact]
        
        batch_grafos = Batch.from_data_list(grafos).to(device)
        Y_batch = Y_batch.to(device)
        
        # Forward 
        outputs = model(batch_grafos)
        loss = criterion(outputs, Y_batch)
        lossi.append(loss.item())

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/20], Loss: 0.2541
Epoch [2/20], Loss: 0.2149
Epoch [3/20], Loss: 0.1818
Epoch [4/20], Loss: 0.1738
Epoch [5/20], Loss: 0.2373
Epoch [6/20], Loss: 0.2070
Epoch [7/20], Loss: 0.2116
Epoch [8/20], Loss: 0.1405
Epoch [9/20], Loss: 0.1591
Epoch [10/20], Loss: 0.1933
Epoch [11/20], Loss: 0.2257
Epoch [12/20], Loss: 0.1252
Epoch [13/20], Loss: 0.0996
Epoch [14/20], Loss: 0.1201
Epoch [15/20], Loss: 0.1643
Epoch [16/20], Loss: 0.1058
Epoch [17/20], Loss: 0.1717
Epoch [18/20], Loss: 0.0997
Epoch [19/20], Loss: 0.1602
Epoch [20/20], Loss: 0.0974


In [83]:
model.batch_size = X_test.size(0)

In [84]:
with torch.no_grad():
    X_to_test = X_test.contiguous().view(seq_len * model.batch_size, n_nodes)
    grafos = [Data(i.view(-1, 1), adj) for i in Xcompact]
        
    batch_grafos = Batch.from_data_list(grafos).to(device)
    predicted = model(batch_grafos)

RuntimeError: shape '[6852, 12, 207]' is invalid for input of size 317952

In [60]:
317952 / 207 /12

128.0

In [53]:
model.batch_size

6852

In [54]:
X_test.view(6852 * 12, -1)

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.